In [1]:
from CBR import *
import numpy as np
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
from statistics import mean

Total Running time 0:00:00


# Experimentació: Explotació de l'Espai de Casos

La idea d'aquest apartat de l'experimentació serà intentar afirmar la següent hipòtesi:

- Si quan un cas és considerat redundant, no comprovem que la valoració sigui millor o pitjor, llavors si generem blocs de casos aleatoris indefinidament, hi haurà un punt on el nombre de casos afegits per bloc serà igual a zero. 

Això serà degut a què tot l'espai de casos ha estat cobert i per tant considera tots els casos com a redundants.

Posteriorment veurem què passa quan sí comprovem si hi ha una millora o no de la valoració. Això farà que s'intercanvïin els casos i, per tant, millori la qualitat de les recomanacions. Alhora trigarà més en arribar el punt on el nombre de casos afegits per bloc sigui zero. A més a més, aquell punt serà en el què el nostre CBR sigui tot el _perfecte_ que les nostres dades ho permeten. 

## Primera Part

In [ ]:
def block(rs:RecommenderSystem, blocksize:int=100):
    
    redundants = 0
    for _ in range(blocksize):
        newp = eval(generate_new_problem())    
        
        most_similar_cases, newcas = rs.retrieve(newp)
        distancies, casos, paths = zip(*most_similar_cases)
        cas_dists = list(zip(distancies, casos))
        dicc_most_sim = rs.reuse(cas_dists, newcas)
        rs.revise(newcas, casos, dicc_most_sim, False) 
        redundancia = rs.retain(newcas, cas_dists, paths[0], 2)
        if redundancia == 1:
            redundants += 1

    return redundants/blocksize

df_problemes = pd.read_csv('./data/problems4000_not_0_0_0.csv')
rs = createRS(
    from_model= False, 
    casebase=df_problemes, 
    maxcasesxleaf=15
)

num_blocks = 100
rates = []

for b in range(num_blocks):
    rate = block(rs)
    rates.append(rate)
    print(f'Rati en el block {b} = {rate}')

plt.plot(range(num_blocks), rates, marker='o')  # 'o' adds dots at each point
plt.xlabel('Quantitat de Blocs Afegits')
plt.ylabel('Rati de Casos Redundants per bloc')
plt.title('Evolució del Rati de Casos Redundants') 
plt.savefig('./images/espaidecasos1')
plt.show()
plt.close() 

## Segona Part

In [ ]:
def block(rs:RecommenderSystem, blocksize:int=100):
    
    afegits = 0
    redundants = 0
    swaps = 0
    for _ in range(blocksize):
        newp = eval(generate_new_problem())    
        
        most_similar_cases, newcas = rs.retrieve(newp)
        distancies, casos, paths = zip(*most_similar_cases)
        cas_dists = list(zip(distancies, casos))
        dicc_most_sim = rs.reuse(cas_dists, newcas)
        rs.revise(newcas, casos, dicc_most_sim, False) 
        redundancia = rs.retain(newcas, cas_dists, paths[0], 2)
        if redundancia == 0:
            afegits += 1
        elif redundancia == 1:
            redundants += 1
        elif redundancia == 2:
            swaps += 1
        rating = sum(newcas.avaluacio)        

    return afegits/blocksize, redundants/blocksize, swaps/blocksize, rating/(blocksize*3)

df_problemes = pd.read_csv('./data/problems4000_not_0_0_0.csv')
rs = createRS(
    from_model= False, 
    casebase=df_problemes, 
    maxcasesxleaf=15
)

num_blocks = 200

afegits, redundants, swaps, ratings = [], [], [], []

for b in range(num_blocks):
    a, re, s, ra = block(rs)
    afegits.append(a)
    redundants.append(re)
    swaps.append(s)
    ratings.append(ra)
    print(f'Mitjana d\'Avaluacions en el block {b} = {a}')

plt.figure(figsize=(12, 6))
plt.plot(range(num_blocks), afegits, label='Afegits')
plt.plot(range(num_blocks), redundants, label='Redundants')
plt.plot(range(num_blocks), swaps, label='Swaps')
plt.xlabel('Quantitat de Blocs Afegits')
plt.ylabel('Valor del Rati per bloc')
plt.title('Evolució dels Ratis de Redundància') 
plt.legend()
plt.savefig('./images/espaidecasos2')
plt.show()
plt.close() 

plt.figure(figsize=(12, 6))  # Ancho de 12 pulgadas y altura de 6 pulgadas
plt.plot(range(num_blocks), ratings, label='Ratings')
plt.xlabel('Quantitat de Blocs Afegits')
plt.ylabel('Rating Mitjà per Bloc')
plt.title('Evolució dels Ratings') 
plt.legend()
plt.savefig('./images/espaidecasos3')
plt.show()
plt.close()
